In [1]:
import cadquery as cq

from jupyter_cadquery import (
    versions,
    show, PartGroup, Part, 
    get_viewer, close_viewer, get_viewers, close_viewers, open_viewer, set_defaults, get_defaults, open_viewer,
    get_pick,
)

from jupyter_cadquery.replay import replay, enable_replay, disable_replay
''
enable_replay(False)


import sys
sys.path.append('../')

Overwriting auto display for cadquery Workplane and Shape

Enabling jupyter_cadquery replay


/home/ben/anaconda3/envs/jcq3/lib/python3.10/site-packages/quaternion/calculus.py:310: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def fd_indefinite_integral(f, t):


In [20]:
from plate import get_plate_shape, get_key_positions, get_key_hole_shape, get_screw_positions, get_keys
from case import add_microcontrollerbox
from config import Config, Shape

from config import read_config_from_json

config_dict = """{
    "name": "Tbaum",
    "row_key_numbers": [
    2,
    8,
    6,
    3,
    1
    ],
    "columnSpacing": 19,
    "rowSpacing": 19,
    "plateThickness": 2,
    "screwHoleDiamater": 2.4,
    "switchHoleSize": 13.9,
    "shape": 0,
    "notched_keyhole": true,
    "caseHeight": 20,
    "caseGap": 0.6,
    "wallThickness": 1.6,
    "floorThickness": 3.0,
    "edgeFillet": 2,
    "bottomFillet": 0,
    "controller": "None",
    "controllerYOffset": 5,
    "controllerBoxThickness": 2
}"""

config_dict = """{
    "name": "square",
    "row_key_numbers": [
        4,
        4,
        4,
        4
    ],
    "columnSpacing": 19,
    "rowSpacing": 19,
    "plateThickness": 2,
    "screwHoleDiamater": 2.4,
    "switchHoleSize": 13.9,
    "shape": 0,
    "notched_keyhole": true,
    "caseHeight": 20,
    "caseGap": 0.6,
    "wallThickness": 2.0,
    "floorThickness": 3.0,
    "edgeFillet": 6,
    "bottomFillet": 2,
    "plateEdgeOffset":-1,
    "controller": {
        "name": "Pi Pico",
        "board_dimension_x": 51.0,
        "board_dimension_y": 21.0,
        "screw_hole_x": 47.0,
        "screw_hole_y": 11.4
    },
    "controllerYOffset": 5,
    "controllerBoxThickness": 2
}"""

config = read_config_from_json(string=config_dict)

In [21]:
def make_plate(config:Config, 
               get_screw_hole_positions:callable=get_screw_positions) -> cq.Sketch:
    
    key_hole_shape = get_key_hole_shape(config)
    kp = get_key_positions(config)
    plate_shape = get_plate_shape(config, kp, config.plateEdgeOffset)
    
    plate = cq.Workplane()
    plate = plate.add(plate_shape).wires().toPending().extrude(config.plateThickness).edges('|Z').fillet(config.edgeFillet)
    plate = plate.cut(get_keys(kp, key_hole_shape, config))

    hole_psns= get_screw_hole_positions(config)
    plate= plate.faces(">Z").workplane().pushPoints(hole_psns).hole(config.screwHoleDiamater)
    return plate


#make_plate(config)

def get_basic_shape(config:Config) -> cq.Sketch:

    kp = get_key_positions(config)
    plate_shape = get_plate_shape(config, kp, config.plateEdgeOffset)
    
    basic_shape_wire= plate_shape.offset2D(config.caseGap, 'intersection')
    
    basic_shape = cq.Workplane().add(basic_shape_wire).wires().toPending().extrude(config.caseHeight)

    basic_shape.edges('>Y').tag('topEdge')
    if config.controller is not None:
        basic_shape = add_microcontrollerbox(basic_shape, config)

    #fillet edges
    basic_shape = basic_shape.edges('|Z').fillet(config.edgeFillet)
    basic_shape.edges('>Z').tag('outerTopEdge') # for making controller box top plate

    return basic_shape



bs = get_basic_shape(config)

bs

CadViewerWidget(anchor=None, cad_width=800, glass=False, height=600, pinning=False, theme='light', title=None,…

In [12]:
thickness =2 
base_fillet = 2
kp = get_key_positions(config)


ps = get_plate_shape(config, kp, -2)

wp = cq.Workplane()
wp=wp.add(ps)
base = wp.wires().toPending().extrude(thickness).edges('|Z').fillet(base_fillet)

base= base.translate((0, 0, 21))
base

CadViewerWidget(anchor=None, cad_width=800, glass=False, height=600, pinning=False, theme='light', title=None,…

In [49]:
cs = get_plate_shape(config, kp, -2).offset2D(config.caseGap, 'intersection')

wp = cq.Workplane()
wp=wp.add(cs)
case = wp.wires().toPending().extrude(20).edges('|Z').fillet(base_fillet)

case= (case.faces(">Z or <Z").shell(config.wallThickness, kind='intersection'))


case.add(base)

100% ⋮————————————————————————————————————————————————————————————⋮ (2/2)  0.08s


CadViewerWidget(anchor=None, cad_width=800, glass=False, height=600, pinning=False, theme='light', title=None,…

In [47]:
case

CadViewerWidget(anchor=None, cad_width=800, glass=False, height=600, pinning=False, theme='light', title=None,…